
# Are There Associations With University Tuition & Professional Pay on a State-by-State Basis?

### Also, How to Conduct Basic Exploratory Data Analysis
Feel free to keep this notebook open, download your own dataset, and follow along with what may be applicable to YOU. <br> Keep in mind, you will almost certainly want to find other ways to manipulate your data. <br> Python has a TON of user created documentation on sites like Stack Overflow.<br> I would recommend searching "[WhatINeedToDoWithMyDataframe], pandas" on Google.

## Table Of Contents:
* [Import Packages and Define Functions](#Import)
* [Loading Data](#Load)
* [Exploring Data](#Explore)
* [Cleaning Data](#Clean)
* [Analyzing Data](#Analyze)
* [Visualizing Data](#Visual)

### Import Packages and Define Functions <a class="anchor" id="Import"></a>
##### You should have a section that imports necessary packages and defines any functions at the top of every notebook, anyone know why?

<details> If packages are not imported before they are used, you will receive errors. 
    <br> It is easiest to 'run all' cells which will go from top to bottom when coming back to your notebook after a break. </details>

##### This section will start almost empty, you will add to it as you import new packages and define function.
- I always recommend starting with 'import pandas as pd' for dataframe management
* Here is the pandas docs - https://pandas.pydata.org/docs/reference/index.html#api
- Otherwise you can ignore these for now

## Loading Data <a class="anchor" id="Load"></a>
- It is best practice keeping this section clean so you know every piece of data you are importing
- You will run into errors if you try and reference data before you have loaded it
- Keeping data in a 'Data' folder is best for organization too

In [0]:
# make sure you have these files
salary_potential = pd.read_csv("./Data/salary_potential.csv")
tuition = pd.read_csv("./Data/tuition_cost.csv")

What does the './' ahead of our folder name do?
<details> Pathing is more complex than you might think. The program does not know where to look for your file. 
    <br> Adding ./ tells the program to start looking from the location where this Notebook is located </details>

## Exploring Our Data <a class="anchor" id="Explore"></a>
- Here is where we want to inform ourselves on the nitty gritty of what information our data actually holds
- How much, what type, is there anything missing, what columns do I need & not need, etc. 

##### Using .head() [first 5 rows] and .tail() [last 5 rows] is a great way to get a first look at our data

In [0]:
salary_potential.head()

In [0]:
tuition.tail()

Sometimes column names have extra spaces or we can't see all of them at first, lets print them and get the shape of each to see how much data we are dealing with. <br>(#Rows, #Columns)

In [0]:
print(salary_potential.columns)
print("")
print(salary_potential.shape)

In [0]:
print(tuition.columns)
print("")
print(tuition.shape)

We also want information on what data TYPE each of the columns are

In [0]:
tuition.info()

We notice that 'state' and 'room_and_board' are missing some data. State_code is not though.

In [0]:
salary_potential.info()

We notice that 'make_world_better_percent' is missing some data, missing data can mess with functions down the road.

## Cleaning and Merging Our Data <a class="anchor" id="Clean"></a>
- Lets get rid of redundant columns, either replace or drop missing values, and do any other cleaning we might want
- Here we will also merge our different data sources into one cohesive one

First up, lets find the entries with no 'state' from tuition

In [0]:
tuition[tuition['state'].isnull()] # null is the value for an empty variable

In [0]:
tuition[tuition['state'].isnull()].state_code.unique()

We can see that there are a few state_codes that don't have state names, we can manually add those.
<br>We'll make a function for this and add it to the top.

In [0]:
def fill_state(row):
    if row['state_code'] == 'AS':
        return 'American Samoa'
    elif row['state_code'] == 'DC':
        return 'District of Columbia'
    elif row['state_code'] == 'PR':
        return 'Puerto Rico'
    elif row['state_code'] == 'GU':
        return 'Guam'
    elif row['state_code'] == 'VI':
        return 'Virgin Islands'
    else:
        return row['state']

# axis = 1 means do it to the columns (0 is the rows)
tuition['state'] = tuition.apply(lambda row : fill_state(row), axis=1) 

Now we check and see there are no more null values!

In [0]:
tuition[tuition['state'].isnull()]

We still have a lot of missing data in the column 'room_and_board' and some in the 'make_world_better_place'
<br><br>
Since we don't care too much about room_and_board, we'll just drop that column as a whole, but for the other, we will just drop the rows that are missing that data since it is only ~3% of our dataset.

In [0]:
# drop room and board (axis = 1 --> column)
tuition = tuition.drop('room_and_board', axis=1)
# drop rows with empty make world better place (axis = 0 --> row)
salary_potential = salary_potential.dropna(subset=['make_world_better_percent'])

In [0]:
# confirm there are no null values in either dataframe, rows or columns (.sum().sum() condences it into 1 value)
print(tuition.isnull().sum().sum(), salary_potential.isnull().sum().sum())

##### Merging our Dataframe
To merge our dataframes we need to identify which columns we want to merge on. There's many ways to combine data, you can dive deeper here: <br>https://realpython.com/pandas-merge-join-and-concat/<br> We will be focusing on a simple merge. <br><br> For a simple merge, we want to have two dataframes that have a column by the same name we want to merge on. <br>In our case we want to merge using the 'name' column. We will only have the data of schools that appear identically in both dataframes.<br> One thing to keep in mind, you must merge on a column with the same name so often will need to rename a column from one dataframe.

In [0]:
# check here to find how to rename - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
# we create a new dataframe called merged_df that is the combined one
merged_df = pd.merge(tuition, salary_potential, on='name')

In [0]:
merged_df.head()

Now we have a 'state' and 'state_name' column that look duplicative, lets check.

In [0]:
merged_df['state'].equals(merged_df['state_name'])

Lets look at the differences, get all the unique 'state' and 'state_names' from all rows where they are not identical

In [0]:
merged_df.loc[~(merged_df['state'] == merged_df['state_name'])]['state'].unique()

In [0]:
merged_df.loc[~(merged_df['state'] == merged_df['state_name'])]['state_name'].unique()

Many are issues with hyphens, some are not, let's fix the hyphen problem and look at the rest.

In [0]:
to_replace = ['New-York', 'North-Carolina', 'South-Dakota', 'South-Carolina',
       'New-Jersey', 'West-Virginia', 'Rhode-Island', 'New-Hampshire',
       'North-Dakota', 'New-Mexico']
replace_with = ['New York', 'North Carolina', 'South Dakota', 'South Carolina',
       'New Jersey', 'West Virginia', 'Rhode Island', 'New Hampshire',
       'North Dakota', 'New Mexico']
for i in range(len(to_replace)):
    merged_df["state_name"].replace({to_replace[i]: replace_with[i]}, inplace=True)
merged_df.loc[~(merged_df['state'] == merged_df['state_name'])]

After searching Manually, Northwestern College is in Iowa & Sterling College is in Kansas. <br> This shows the importance of double checking your data. <br> I will drop the 'state column' as the 'state_name' column is the accurate one and state_code is unnecessary.

In [0]:
# inplace = True means that we want the rename to occur in existing dataframe, not return a new one
merged_df.drop(['state','state_code'], axis=1, inplace=True)
merged_df.head()

In [0]:
merged_df.shape

Now that we're done cleaning and merging our data, let's turn that into a function, and add it to the top as well to keep our notebook organized.

In [0]:
# this function cleans and merges our two dataframes, returns one merged_df
def clean_and_merge_dfs(tuition, salary_potential):
    # drop room and board (axis = 1 --> column)
    tuition = tuition.drop('room_and_board', axis=1)
    # drop rows with empty make world better place (axis = 0 --> row)
    salary_potential = salary_potential.dropna(subset=['make_world_better_percent'])
    # we create a new dataframe called merged_df that is the combined one
    merged_df = pd.merge(tuition, salary_potential, on='name')
    # replace mismatched rows
    to_replace = ['New-York', 'North-Carolina', 'South-Dakota', 'South-Carolina',
       'New-Jersey', 'West-Virginia', 'Rhode-Island', 'New-Hampshire',
       'North-Dakota', 'New-Mexico']
    replace_with = ['New York', 'North Carolina', 'South Dakota', 'South Carolina',
           'New Jersey', 'West Virginia', 'Rhode Island', 'New Hampshire',
           'North Dakota', 'New Mexico']
    for i in range(len(to_replace)):
        merged_df["state_name"].replace({to_replace[i]: replace_with[i]}, inplace=True)
    # drop the duplicative (and slightly incorrect) column 'state'
    merged_df.drop(['state','state_code'], axis=1, inplace=True)
    # return the merged dataframe
    return merged_df

## Analyzing Our Data <a class="anchor" id="Analyze"></a>
Here we want to start looking at our data in the context of our original goal. <br> "Are There Associations With University Tuition & Professional Pay on a State-by-State Basis?"<br> To do this I'm going to manually create a dataset, getting first the number of schools in that state, and then the average of statistics I am investigating.

In [0]:
def state_stats_df(merged_df):
    # get a list of all state_name's
    states = merged_df.state_name.unique()

    # Create a column in a new dataframe for each stat I want
    state_stats = pd.DataFrame(index=states, columns=['number of schools','average early_career_pay',
                                                      'average mid_career_pay','average make_world_better_percent',
                                                      'average stem_percent','average in_state_tuition','average out_of_state_tuition'])
    # Fill in the columns for each state
    for state in states:
        state_stats.loc[state, 'number of schools'] = len(merged_df[merged_df.state_name == state])
        state_stats.loc[state, 'average early_career_pay'] = merged_df[merged_df.state_name == state]['early_career_pay'].mean()
        state_stats.loc[state, 'average mid_career_pay'] = merged_df[merged_df.state_name == state]['mid_career_pay'].mean()
        state_stats.loc[state, 'average make_world_better_percent'] = merged_df[merged_df.state_name == state]['make_world_better_percent'].mean()
        state_stats.loc[state, 'average stem_percent'] = merged_df[merged_df.state_name == state]['stem_percent'].mean()
        state_stats.loc[state, 'average in_state_tuition'] = merged_df[merged_df.state_name == state]['in_state_tuition'].mean()
        state_stats.loc[state, 'average out_of_state_tuition'] = merged_df[merged_df.state_name == state]['out_of_state_tuition'].mean()
    # sort alphabetically by state
    state_stats.sort_index(inplace = True)
    return state_stats

state_stats = state_stats_df(merged_df)
state_stats.head()

We don't need to stay with just the data we have been given, we can combine columns or do additional calculations on them. <br> One example is say we want to get the average 4 year cost of tuition, we can create those columns as well (and a few others)

In [0]:
def add_details(state_stats):
    # convert all columns to numeric
    state_stats = state_stats.apply(pd.to_numeric)
    # make columns for four year average out of state and in state total
    state_stats['four year average out of state tuition'] = state_stats['average out_of_state_tuition'] * 4
    state_stats['four year average in state tuition'] = state_stats['average in_state_tuition'] * 4
    # make column calculating how many years on average to repay four year average in and our of state based on average early career pay in index 0
    state_stats['years to repay in state 4 year uni based on early career pay'] =  state_stats['four year average in state tuition'] / state_stats['average early_career_pay']
    state_stats['years to repay out of state 4 year uni based on early career pay'] = state_stats['four year average out of state tuition'] / state_stats['average early_career_pay'] 
    # make column calculating how many years on average to repay four year average in and our of state based on average mid career pay in index 0
    state_stats['years to repay in state 4 year uni based on mid career pay'] =  state_stats['four year average in state tuition'] / state_stats['average mid_career_pay']
    state_stats['years to repay out of state 4 year uni based on mid career pay'] = state_stats['four year average out of state tuition'] / state_stats['average mid_career_pay'] 
    # move 'years to repay out of state 4 year uni based on early career pay' to the front of the dataframe
    state_stats = state_stats.reindex(columns=['four year average in state tuition', 'four year average out of state tuition', 'years to repay in state 4 year uni based on early career pay', 'years to repay out of state 4 year uni based on early career pay',
    'years to repay in state 4 year uni based on mid career pay', 'years to repay out of state 4 year uni based on mid career pay',
    'number of schools', 'average early_career_pay', 'average mid_career_pay', 'average make_world_better_percent', 'average stem_percent',
    'average in_state_tuition','average out_of_state_tuition'])
    return state_stats

In [0]:
state_stats = add_details(state_stats)
state_stats.head()

Notice: This assumes you divert your entire paycheck to student loans and there is no interest. While this gives a guidline, that does hurt the usefullness, as different areas may allow to divert different amounts. Look out for this type of error in your own work

One of my favorite ways to investigate a dataframe is by using highlighting. <br> Here is a quick example of how to highlight based on how the value compares to the average. <br>
These functions are a bit confusing because they are meant to be used by the pandas Apply function. <br> Read more here: https://www.datacamp.com/community/tutorials/pandas-apply <br> It is a way to check each row and do 'something' based on some values.

This uses python's "list comprehension", which you can find more on here - https://www.w3schools.com/python/python_lists_comprehension.asp

In [0]:
# Highlighting functions for our dataframe
def highlight_above_avg(state):
    # state_TF will check every row (state) and give a True/False value depending on the condition asked
    # e.g. if the value is >.50 quantile (average value) for the column
    state_TF = state >= state.quantile(.50)
    return ['background: lightgreen' if state else '' for state in state_TF]
def highlight_top(s):   
    state_TF = s > s.quantile(0.90)
    return ['color: blue' if state else '' for state in state_TF]

def return_highlighted_df(state_stats):
    return state_stats.style.apply(highlight_above_avg).apply(highlight_top)

Add any other filters you can think of to the filters

Something to note aboout the below table, some green / blue values could be 'good' while others could be 'bad'. <br>
E.g. Having a top 10% in 'average_early_career_pay' could be good, but having a top 10% 'years to repay out of state 4 year uni based on early career pay' would be less awesome.

In [0]:
return_highlighted_df(state_stats)

## Visualizing Our Data <a class="anchor" id="Visual"></a>
Last but not least we want to visualize our data, you can see different Python visualization libraries compared here:https://towardsdatascience.com/top-6-python-libraries-for-visualization-which-one-to-use-fe43381cd658 <br> We are using Plotly, as it is the most powerful and simple out of the box. <br> You can see some of the capabilities of Plotly in their documentation here: https://plotly.com/python/ <br> Note: if while running this you get an error 'PX IS NOT DEFINED' you did not import plotly.express (replace plotly with plotly.express in the import).

First things first, now that we have completed our calculations we will turn the index into a named column for ease of graphing and we need to include state code if we want to graph on a map.


In [0]:
# turn the index into a column
state_stats_extras = state_stats.reset_index(level=0)
# rename index to state
state_stats_extras.rename({'index': 'state'}, axis=1, inplace=True)
# finds each state and adds the code
state_stats_extras = state_stats_extras.merge(tuition[['state','state_code']], on='state').drop_duplicates().reset_index(drop=True)
state_stats_extras.head()

In [0]:
# Define a function to add these last extras that need to be applied after our numerical calculations (highlighting for example)
# Don't forget to pass in tuition as a parameter because you merge with it
def add_extras(state_stats,tuition):
    # turn the index into a column
    state_stats_extras = state_stats.reset_index(level=0)
    # rename index to state
    state_stats_extras.rename({'index': 'state'}, axis=1, inplace=True)
    # add the state_code column
    state_stats_extras = state_stats_extras.merge(tuition[['state','state_code']], on='state').drop_duplicates().reset_index(drop=True)
    return state_stats_extras

In [0]:
fig = px.scatter(state_stats_extras, x='four year average in state tuition', y='average mid_career_pay',
                 color='state',size='average make_world_better_percent',
                 trendline="ols", trendline_scope = 'overall',
                 title= 'Average In State Tuition vs Average % of Students Graduating with STEM Degrees')

fig.show()

In [0]:
fig = px.scatter(state_stats_extras, x='average stem_percent', y='average early_career_pay',color='state',
                 trendline="ols", trendline_scope = 'overall',
                 title= 'Average In State Tuition vs Average % of Students Graduating with STEM Degrees')

fig.show()

In [0]:
fig = px.choropleth(state_stats_extras, locations='state_code', locationmode="USA-states", color="four year average in state tuition", scope="usa")
fig.show()

In [0]:
fig = px.choropleth(state_stats_extras, locations='state_code', locationmode="USA-states", color="average early_career_pay", scope="usa")
fig.show()

Add any more ideas you have here: